In [17]:
import pandas as pd
import numpy as np
import sqlite3

In [19]:
# Connect to the db and query for the time column

try:
    conn = sqlite3.connect('../BTC_data.db')
    query = "SELECT time FROM BTC_data;"
    df = pd.read_sql_query(query, conn)
except sqlite3.Error as e:
    print(f"An error occurred: {e}")
finally:
    if conn:
        conn.close()


In [20]:
# Feature Engineering

df['time'] = pd.to_datetime(df['time'])
df['hour'] = df['time'].dt.hour
df['day_of_week'] = df['time'].dt.dayofweek  # Monday=0, Sunday=6

# Define opening hours for USA (9-17 UTC), EU (8-16 UTC), ASIA (1-9 UTC)
df['USA_open'] = (df['hour'] >= 9) & (df['hour'] < 17)
df['EU_open'] = (df['hour'] >= 8) & (df['hour'] < 16)
df['ASIA_open'] = (df['hour'] >= 1) & (df['hour'] < 9)


In [21]:
df.head()

,time,hour,day_of_week,USA_open,EU_open,ASIA_open
0,2022-09-01 06:45:00,6,3,False,False,True
1,2022-09-01 07:00:00,7,3,False,False,True
2,2022-09-01 07:15:00,7,3,False,False,True
3,2022-09-01 07:30:00,7,3,False,False,True
4,2022-09-01 07:45:00,7,3,False,False,True


# Append back to the db

try:
    conn = sqlite3.connect('../BTC_data.db')
    # Append new columns only if they don't exist
    cursor = conn.cursor()
    cursor.execute("PRAGMA table_info(BTC_data);")
    existing_columns = [column[1] for column in cursor.fetchall()]
    for col in ['hour', 'day_of_week', 'USA_open', 'EU_open', 'ASIA_open']:
        if col not in existing_columns:
            conn.execute(f"ALTER TABLE BTC_data ADD COLUMN {col} INTEGER;")
    conn.commit()

    # Update the new columns
    for index, row in df.iterrows():
        update_query = f"""
        UPDATE BTC_data
        SET hour = {row['hour']},
            day_of_week = {row['day_of_week']},
            USA_open = {int(row['USA_open'])},
            EU_open = {int(row['EU_open'])},
            ASIA_open = {int(row['ASIA_open'])}
        WHERE time = "{row['time']}";
        """
        conn.execute(update_query)
    conn.commit()
    
    # Verify that the update is successful
    df_check = pd.read_sql_query("SELECT * FROM BTC_data LIMIT 5;", conn)
    print(df_check)

except sqlite3.Error as e:
    print(f"An error occurred: {e}")
    conn.rollback()
finally:
    if conn:
        conn.close()



In [24]:
# Studying what other features we could add to the data from itself

df.columns

Index(['time', 'hour', 'day_of_week', 'USA_open', 'EU_open', 'ASIA_open'], dtype='object')